# Empirical Data Exploratory Analysis

This notebook contains exploratory data analysis for empirical data.

## Import libraries

In [7]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import re
import csv
import io


## Load data

In [11]:
import re
import csv
import io

# --- Configuration ---
# Define the path to your Fama-French data file
file_path = "../../Data/Fama_french/100_Portfolios_10x10.CSV" # Using path provided by user

# Keywords often found in header/marker lines preceding data blocks
keywords = [
    "Average", "Returns", "Number of Firms", "Sum of",
    "Value Weight", "Equal Weight", "Size", "BE/ME",
    "Monthly", "Annual", "Portfolio", "Portfolios"
]
# Minimum number of commas to guess if a line might be CSV data/header
min_commas_for_header = 5 # Threshold for headers
min_commas_for_data = 3   # Threshold for data

# --- Helper Functions ---
def line_looks_like_data(line_stripped, min_commas):
    """Checks if a line looks like a typical data row (starts numeric, has commas)."""
    if not line_stripped:
        return False
    parts = line_stripped.split(',')
    if not parts: return False # Handle empty lines split
    try:
        float(parts[0].strip())
        is_first_part_numeric = True
    except ValueError:
        is_first_part_numeric = False
    return is_first_part_numeric and line_stripped.count(',') >= min_commas

def line_looks_like_header(line_stripped, min_commas):
    """Checks if a line looks like a CSV header row (many commas, maybe non-numeric start)."""
    if not line_stripped:
        return False
    # A header often has many commas
    has_enough_commas = line_stripped.count(',') >= min_commas
    # Optional: Check if it likely contains non-numeric characters typical of headers
    has_text = bool(re.search(r'[a-zA-Z]', line_stripped))
    # Optional: Check if it *doesn't* look like a standard data row
    # is_data = line_looks_like_data(line_stripped, min_commas_for_data) # Avoid circularity if thresholds differ

    # Primarily rely on comma count and presence of text
    return has_enough_commas and has_text # and not is_data

# --- Function to Identify Blocks ---
def identify_potential_blocks(filepath, keywords_list, min_commas_hdr, min_commas_data):
    """Scans a file line by line to identify potential data block start points."""
    potential_blocks = []
    lines = []
    try:
        with io.open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()
    except UnicodeDecodeError:
        print("UTF-8 encoding failed, trying latin1...")
        with io.open(filepath, 'r', encoding='latin1') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return []
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return []

    identified_lines = set() # Keep track of lines already marked as start points

    # Iterate through lines, looking for potential markers
    for i, line in enumerate(lines):
        line_stripped = line.strip()

        if not line_stripped or i in identified_lines:
            continue

        # --- Heuristic 1: Standard Title (keywords + '--') followed by data/header ---
        contains_keywords = any(keyword.lower() in line_stripped.lower() for keyword in keywords_list)
        looks_like_std_title = contains_keywords and "--" in line_stripped
        next_line_looks_ok = False
        if i + 1 < len(lines):
            next_line_stripped = lines[i+1].strip()
            if next_line_stripped:
                 next_line_looks_ok = line_looks_like_data(next_line_stripped, min_commas_data) or \
                                      line_looks_like_header(next_line_stripped, min_commas_hdr)

        if looks_like_std_title and next_line_looks_ok:
            if i not in identified_lines:
                 # Check if the previous identified block wasn't the immediately preceding line
                 is_new_block = True
                 if potential_blocks:
                     last_block_line = potential_blocks[-1]['line_number']
                     if i - last_block_line <= 2: # Allow for a blank line gap perhaps
                         is_new_block = False

                 if is_new_block:
                    potential_blocks.append({
                        "line_number": i,
                        "marker_text": line_stripped,
                        "reason": "Standard Title (--)"
                    })
                    identified_lines.add(i)
                    continue # Prioritize this type of match

        # --- Heuristic 2: Header line found, look backwards for description ---
        is_header = line_looks_like_header(line_stripped, min_commas_hdr)
        if is_header:
            # Look back up to 5 lines for the first non-blank, non-data line containing keywords
            found_description = False
            for j in range(i - 1, max(-1, i - 6), -1): # Look back from i-1 down to i-5
                if j < 0: break # Stop if we reach beginning of file
                prev_line_stripped = lines[j].strip()

                if not prev_line_stripped: # Skip blank lines
                    continue

                # Check if this previous line looks like a description
                prev_contains_keywords = any(keyword.lower() in prev_line_stripped.lower() for keyword in keywords_list)
                prev_is_data = line_looks_like_data(prev_line_stripped, min_commas_data)

                if prev_contains_keywords and not prev_is_data:
                    # Found a likely description line! Mark line j as the start.
                    # Avoid adding if j is too close to the previously identified block start
                    if j not in identified_lines:
                        is_new_block = True
                        if potential_blocks:
                            last_block_line = potential_blocks[-1]['line_number']
                            # Check if this found description line 'j' is sufficiently after the last block
                            if j - last_block_line <= 5: # Heuristic: Allow some gap, but not too close
                                # Also check if the header 'i' is very close to the last identified line
                                if i - last_block_line <= 5:
                                     is_new_block = False # Likely part of the previous block's header/desc

                        if is_new_block:
                            potential_blocks.append({
                                "line_number": j, # Mark the description line
                                "marker_text": prev_line_stripped,
                                "reason": f"Description Found Before Header on Line {i+1}"
                            })
                            identified_lines.add(j)
                            found_description = True
                            break # Stop looking back once description found

            # If we found a description by looking back, continue to next line 'i'
            # This prevents the header line 'i' itself from being flagged by other heuristics later
            if found_description:
                 identified_lines.add(i) # Mark header line as processed too
                 continue


    # --- Post-processing: Sort results ---
    potential_blocks.sort(key=lambda x: x['line_number'])

    # Filter out blocks starting too close to each other (can happen with multi-line headers/titles)
    final_blocks = []
    last_line = -10 # Initialize far back
    for block in potential_blocks:
        if block['line_number'] - last_line > 2: # Only keep if sufficiently spaced from the last kept block
             final_blocks.append(block)
             last_line = block['line_number']
        # else:
            # print(f"Filtering out block at line {block['line_number']+1} due to proximity to previous.")


    return final_blocks

# --- Main Execution ---
print(f"Scanning '{file_path}' for potential data block markers (v3)...")
blocks = identify_potential_blocks(file_path, keywords, min_commas_for_header, min_commas_for_data)

if blocks:
    print("\nFound potential data block starting points:")
    for block in blocks:
        # Add 1 to line_number for conventional 1-based line counting for display
        print(f"  Line {block['line_number'] + 1}: {block['marker_text']} (Reason: {block['reason']})")
    print("\nNote: These are potential markers. Please review the file around these lines.")
    print("The actual data usually starts on the line *after* the marker, or sometimes after the header line following the marker.")
else:
    print("Could not automatically identify distinct data blocks based on the v3 heuristics.")
    print("You may need to manually inspect the file to find the section headers.")


Scanning '../../Data/Fama_french/100_Portfolios_10x10.CSV' for potential data block markers (v3)...

Found potential data block starting points:
  Line 15: Average Value Weighted Returns -- Monthly (Reason: Standard Title (--))
  Line 1201: Average Equal Weighted Returns -- Monthly (Reason: Standard Title (--))
  Line 2387: Average Value Weighted Returns -- Annual (Reason: Standard Title (--))
  Line 2489: Average Equal Weighted Returns -- Annual (Reason: Standard Title (--))
  Line 2591: Number of Firms in Portfolios (Reason: Description Found Before Header on Line 2592)
  Line 3777: Average Market Cap (Reason: Description Found Before Header on Line 3778)
  Line 4964: Value Weight Average of BE/ME Calculated for June of t to June of t+1 as: (Reason: Description Found Before Header on Line 4968)
  Line 6154: Value Weight Average of BE_FYt-1/ME_June t Calculated for June of t to June of t+1 as: (Reason: Description Found Before Header on Line 6158)
  Line 7344: Value Weight Average of 

In [12]:
# --- Configuration ---
file_path = '../../Data/Fama_french/100_Portfolios_10x10.CSV' # Using path provided by user
missing_value = -99.99

# Define the blocks to load based on the identifier script's output
# Format: block_name: (marker_line_0_indexed, next_block_marker_line_0_indexed)
# Use the line numbers where the *marker text* was found.
# The header is assumed to be on the line *after* the marker line for standard titles,
# or identified explicitly by the look-back heuristic.
block_info = {
    "VW_Returns_Monthly": (14, 1200), # Marker line 14 ("Avg VW Returns -- Monthly"), next marker line 1200 ("Avg EW Returns -- Monthly")
    "EW_Returns_Monthly": (1200, 2386), # Marker line 1200, next marker line 2386 ("Avg VW Returns -- Annual")
    "Num_Firms": (2590, 3776), # Marker line 2590 ("Num Firms..."), next marker line 3776 ("Avg Mkt Cap...")
    "Avg_Mkt_Cap": (3776, 4963) # Marker line 3776, next marker line 4963 ("VW Avg BE/ME...")
    # Note: Using line numbers (0-indexed) of the marker lines identified previously.
    # The end line for the last block (Avg_Mkt_Cap) is the line before the next marker (4964-1 = 4963).
}

# --- Function to load and clean a specific block ---
def load_fama_french_block(filepath, block_name, marker_line, next_marker_line, null_val):
    """Loads a specific block of Fama-French data using Polars."""
    try:
        # Calculate skip_rows: Skip lines up to and including the marker line.
        # The header is expected on the line immediately after the marker line.
        # Example: If marker is line 14 (0-indexed), header is line 15. skip_rows should be 15.
        rows_to_skip = marker_line + 1

        # Calculate n_rows: Number of data rows between the current header and the next marker.
        # header_line = marker_line + 1
        # data_start_line = header_line + 1
        # data_end_line = next_marker_line - 1
        # n_rows = data_end_line - data_start_line + 1 = (next_marker_line - 1) - (marker_line + 1 + 1) + 1
        # n_rows = next_marker_line - marker_line - 2
        # However, polars n_rows reads *at most* n_rows *after* skipping rows *before* the header.
        # If header is at rows_to_skip, data starts at rows_to_skip + 1.
        # We want to read up to line next_marker_line - 1.
        # Total lines in block = next_marker_line - rows_to_skip
        # Number of data rows = Total lines in block - 1 (for header) = next_marker_line - rows_to_skip - 1
        n_rows_to_read = next_marker_line - rows_to_skip

        print(f"Loading block '{block_name}':")
        print(f"  Marker Line (0-idx): {marker_line}")
        print(f"  Header Line (0-idx): {rows_to_skip}")
        print(f"  Next Marker Line (0-idx): {next_marker_line}")
        print(f"  Calculated skip_rows: {rows_to_skip}")
        print(f"  Calculated n_rows: {n_rows_to_read}")


        df = pl.read_csv(
            filepath,
            skip_rows=rows_to_skip, # Skip lines *before* the header
            n_rows=n_rows_to_read,  # Read this many rows *after* skipping
            has_header=True,        # The first row read (after skipping) is the header
            null_values=str(null_val),
            separator=",",
            ignore_errors=True,     # Try to ignore rows that cause parsing errors
            encoding='utf-8'
        )

        # --- Data Cleaning ---
        # Rename the first column (date) - often unnamed or 'Unnamed: 0'
        date_col_name = df.columns[0]
        df = df.rename({date_col_name: "Date"})

        # Convert Date column to string first to handle potential mixed types/whitespace
        df = df.with_columns(pl.col("Date").cast(pl.Utf8))

        # Filter out rows where 'Date' is not numeric (removes potential footers/text)
        df = df.filter(pl.col("Date").is_not_null() & pl.col("Date").str.contains(r"^\s*\d+\s*$"))

        # Convert Date column to actual Date type (YYYYMM format)
        # Use strict=False to handle potential parsing errors gracefully (results in null)
        df = df.with_columns(
            pl.col("Date").str.strip_chars().str.strptime(pl.Date, format="%Y%m", strict=False).alias("Date")
        )

        # Convert all other columns to Float64 (suitable for returns, market cap, and can handle NaNs in Num Firms)
        value_columns = df.columns[1:] # Exclude the 'Date' column
        df = df.with_columns(
            [pl.col(c).str.strip_chars().cast(pl.Float64, strict=False) for c in value_columns]
        )

        # Drop rows with invalid dates (NaT) or where all values are null
        df = df.drop_nulls(subset=["Date"])
        df = df.filter(pl.sum_horizontal(pl.all().is_not_null()) > 1) # Keep rows with Date + at least one value

        # Optional: Convert returns from percent to decimal (uncomment if needed)
        # if "Returns" in block_name:
        #     print(f"  Converting returns in '{block_name}' to decimals...")
        #     for col_name in value_columns:
        #         df = df.with_columns((pl.col(col_name) / 100.0).alias(col_name))

        # Set Date as index (Polars doesn't have a direct index like pandas, but sorting helps)
        df = df.sort("Date")

        print(f"  Successfully loaded and processed block '{block_name}'. Shape: {df.shape}")
        return df

    except pl.NoDataError:
        print(f"Warning: No data found for block '{block_name}'. Check markers and file structure.")
        return None
    except Exception as e:
        print(f"An error occurred processing block '{block_name}': {e}")
        import traceback
        traceback.print_exc() # Print detailed traceback for debugging
        return None

# --- Main Execution ---
loaded_data = {}
blocks_to_load = [
    "VW_Returns_Monthly",
    "EW_Returns_Monthly",
    "Num_Firms",
    "Avg_Mkt_Cap"
]

for block_name in blocks_to_load:
    if block_name in block_info:
        marker_line, next_marker_line = block_info[block_name]
        try:
            # Attempt to load using utf-8 first
            df = load_fama_french_block(file_path, block_name, marker_line, next_marker_line, missing_value)
            if df is None and isinstance(e, UnicodeDecodeError): # Check if loading failed due to encoding
                 raise UnicodeDecodeError # Re-raise to trigger latin1 attempt
            loaded_data[block_name] = df
        except UnicodeDecodeError:
             # If utf-8 fails, try latin1
            print(f"UTF-8 failed for {block_name}, trying latin1...")
            try:
                 df = load_fama_french_block(file_path.replace('utf-8', 'latin1'), block_name, marker_line, next_marker_line, missing_value)
                 loaded_data[block_name] = df
            except Exception as e_latin1:
                 print(f"Latin1 encoding also failed for block '{block_name}': {e_latin1}")
                 loaded_data[block_name] = None
        except Exception as e:
             # Catch other potential errors during loading
             print(f"Failed to load block '{block_name}' due to error: {e}")
             loaded_data[block_name] = None # Ensure key exists even if loading fails
    else:
        print(f"Warning: Block information for '{block_name}' not found.")
        loaded_data[block_name] = None


# --- Display Results ---
print("\n--- Loaded Data Summary ---")
for block_name, df in loaded_data.items():
    print(f"\n--- {block_name} ---")
    if df is not None:
        print(df.head())
        # print(df.describe()) # Uncomment for summary stats
    else:
        print("Failed to load.")



Loading block 'VW_Returns_Monthly':
  Marker Line (0-idx): 14
  Header Line (0-idx): 15
  Next Marker Line (0-idx): 1200
  Calculated skip_rows: 15
  Calculated n_rows: 1185
  Successfully loaded and processed block 'VW_Returns_Monthly'. Shape: (1182, 101)
Loading block 'EW_Returns_Monthly':
  Marker Line (0-idx): 1200
  Header Line (0-idx): 1201
  Next Marker Line (0-idx): 2386
  Calculated skip_rows: 1201
  Calculated n_rows: 1185
  Successfully loaded and processed block 'EW_Returns_Monthly'. Shape: (1182, 101)
Loading block 'Num_Firms':
  Marker Line (0-idx): 2590
  Header Line (0-idx): 2591
  Next Marker Line (0-idx): 3776
  Calculated skip_rows: 2591
  Calculated n_rows: 1185
  Successfully loaded and processed block 'Num_Firms'. Shape: (1182, 101)
Loading block 'Avg_Mkt_Cap':
  Marker Line (0-idx): 3776
  Header Line (0-idx): 3777
  Next Marker Line (0-idx): 4963
  Calculated skip_rows: 3777
  Calculated n_rows: 1186
  Successfully loaded and processed block 'Avg_Mkt_Cap'. Shape